In [1]:
import wandb
import os
import pandas as pd
import numpy as np
import json

import matplotlib.pyplot as plt

from PIL import Image

In [2]:
api = wandb.Api()

In [55]:
def get_runs(runname, project="pasqualedem/weedmapping-rededge"):
    # Project is specified by <entity/project-name>
    return api.runs(project, filters={'display_name':runname})


def get_table(run):
    arts = run.logged_artifacts()
    seg = [a for a in arts if "test_seg" in a.name][0]
    seg.download()
    root = seg._download_roots.pop()
    table_json = json.load(open(os.path.join(root, "test_seg.table.json")))
    table = pd.DataFrame(table_json["data"], columns=table_json["columns"])
    table["Predictions"] = table["Image"].apply(lambda x: x['masks']['predictions']['path'])
    table["Ground Truth"] = table["Image"].apply(lambda x: x['masks']['ground_truth']['path'])
    table["Image"] = table["Image"].apply(lambda x: x['path'])
    return table, root

def get_image_pred_truth(root, row):
    name = row["ID"]
    img = np.array(Image.open(os.path.join(root, row["Image"])))
    pred = np.array(Image.open(os.path.join(root, row["Predictions"])))
    truth = np.array(Image.open(os.path.join(root, row["Ground Truth"])))
    pred = convert_to_rgb(pred)
    truth = convert_to_rgb(truth)
    return name, img, pred, truth

def convert_to_rgb(seg):
    cmap = {0: (0, 0, 0), 1: (0, 255, 0), 2: (255, 0, 0)}
    seg_rgb = np.zeros((seg.shape[0], seg.shape[1], 3), dtype=np.uint8)
    for k, v in cmap.items():
        seg_rgb[seg == k] = v
    return seg_rgb

def plot_range(table, root, start, end):
    for i in range(start, end):
        name, img, pred, truth = get_image_pred_truth(root, table.iloc[i])
        plot_image_pred_truth(name, img, pred, truth)

def plot_image_pred_truth(name, img, pred, truth):
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
    ax[0].imshow(img)
    ax[0].set_title("Image")
    ax[1].imshow(pred)
    ax[1].set_title("Predictions")
    ax[2].imshow(truth)
    ax[2].set_title("Ground Truth")
    plt.suptitle(name)
    plt.show()
        
def get_image_pred_truth_by_names(root, table, names):
    rows = table[table["ID"].isin(names)]
    if len(rows) == 0:
        table['ID'] = table['ID'].apply(lambda x: "test_seg_" + x)
        rows = table[table["ID"].isin(names)]
    fun = lambda x: Image.fromarray(convert_to_rgb(np.array(Image.open(os.path.join(root, x)))))
    rows['Image'] = rows['Image'].apply(lambda x: Image.open(os.path.join(root, x)))
    rows['Predictions'] = rows['Predictions'].apply(fun)
    rows['Ground Truth'] = rows['Ground Truth'].apply(fun)
    return rows

def save_image_pred_truth(root, destination, table, names, file_name):
    rows = get_image_pred_truth_by_names(root, table, names)
    for row in rows.iterrows():
        idx, (name, img, pred, truth) = row
        name, ext = os.path.splitext(name)
        img.save(os.path.join(destination, name + "_img.png"))
        pred.save(os.path.join(destination, file_name + name + "_pred.png"))
        truth.save(os.path.join(destination, name + "_truth.png"))

def save_image_pred_truth_of_run(runname, plot_names, file_name, project="pasqualedem/weedmapping-rededge", destination="images"):
    runs = get_runs(runname, project)
    for run in runs:
        table, root = get_table(run)
        save_image_pred_truth(root, destination, table, plot_names, file_name)

---

In [33]:
run = get_runs("hearty-plant-115", project="pasqualedem/weedmapping-rededge")[0]

In [34]:
table, root = get_table(run)

wandb:   1048 of 1048 files downloaded.  


In [ ]:
plot_range(table, root, 0, 3)

---

In [56]:
selected_plots = [
    "test_seg_003_frame0148_1.png",
    "test_seg_003_frame0111_2.png",
    "test_seg_003_frame0136_0.png",
    "test_seg_003_frame0136_3.png",
    "test_seg_003_frame0134_3.png",
]

In [57]:
destination = "images"
os.makedirs(destination, exist_ok=True)
save_image_pred_truth_of_run("hearty-plant-115", plot_names=selected_plots, file_name="SplitLawin_", destination=destination, project="pasqualedem/weedmapping-rededge")

wandb:   1048 of 1048 files downloaded.  
/tmp/ipykernel_401899/4054073006.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rows['Image'] = rows['Image'].apply(lambda x: Image.open(os.path.join(root, x)))
/tmp/ipykernel_401899/4054073006.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rows['Predictions'] = rows['Predictions'].apply(fun)
/tmp/ipykernel_401899/4054073006.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

In [58]:
destination = "images"
os.makedirs(destination, exist_ok=True)
save_image_pred_truth_of_run("jumping-haze-94", plot_names=selected_plots, file_name="DoubleLawin_", destination=destination, project="pasqualedem/weedmapping-rededge")

wandb:   1051 of 1051 files downloaded.  
/tmp/ipykernel_401899/4054073006.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rows['Image'] = rows['Image'].apply(lambda x: Image.open(os.path.join(root, x)))
/tmp/ipykernel_401899/4054073006.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rows['Predictions'] = rows['Predictions'].apply(fun)
/tmp/ipykernel_401899/4054073006.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

In [59]:
destination = "images"
os.makedirs(destination, exist_ok=True)
save_image_pred_truth_of_run("cosmic-surf-16", plot_names=selected_plots, file_name="Lawin_", destination=destination, project="pasqualedem/weedmapping-rededge")

wandb:   1055 of 1055 files downloaded.  
/tmp/ipykernel_401899/4054073006.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rows['Image'] = rows['Image'].apply(lambda x: Image.open(os.path.join(root, x)))
/tmp/ipykernel_401899/4054073006.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rows['Predictions'] = rows['Predictions'].apply(fun)
/tmp/ipykernel_401899/4054073006.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i